## 4. Predicción de fugas de clientes en una compañía telefónica (3.5 puntos)

Una compañía telefónica está interesada en que desarrollemos un modelo que prediga los **100 clientes actuales** (dataset de explotación) que tienen más probabilidad de abandonar la compañía. Para ello nos proporcionan una base de datos **fuga_clientes_empresa_telefonica_construccion.csv** (en carpeta data/) con casos etiquetados, que usaremos para construir nuestro modelo de predicción.

Los campos de esta base de datos son:

* **Customer ID**

* **network_age**: antigüedad del cliente en días

* **Customer tenure in months:** antigüedad del cliente en meses

* **Total Spend in Months 1 and 2:** gasto total del cliente en los meses de referencia 1 y 2

* **Total SMS Spend:** gasto total en SMS

* **Total Data Spend:** gasto total en datos/internet

* **Total Data Consumption:** consumo total de datos (en KB) durante el período de estudio

* **Total Unique Calls:** número total de llamadas únicas

* **Total Onnet spend:** gasto total en llamadas a otros usuarios de la misma red de telefonía

* **Total Offnet spend:** gasto total en llamadas a otros usuarios de redes de telefonía diferentes

* **Total Call centre complaint calls:** número de llamadas de queja al call center

* **Network type subscription in Month 1:** suscripción de tipo de red en el mes 1. Esto indica la suscripción de red preferida de un cliente, lo que puede indicar su tipo de dispositivo: servicio 2G o 3G

* **Network type subscription in Month 2:** igual que el anterior pero en el mes posterior

* **Churn Status**: el valor es 1 si el cliente abandona la compañía telefónica, 0 si permanece en ella

* **Most Loved Competitor network in Month 1:** qué otro proveedor de la competencia prefiere el cliente en el mes 1. En realidad es un conjunto de columnas, cada una enfocada en un proveedor particular

* **Most Loved Competitor network in Month 2:** qué otro proveedor de la competencia prefiere el cliente en el mes 2. En realidad es un conjunto de columnas, cada una enfocada en un proveedor particular

La variable a predecir es **Churn Status**: el valor es 1 si el cliente **abandona** la compañía, 0 si no la abandona.

La compañía también nos proporciona otra base de datos, **fuga_clientes_empresa_telefonica_explotacion.csv**, con información sobre clientes de los que no se sabe ahora mismo si van a permanecer o no en la compañía. Por tanto en esta segunda base de datos todos los valores de la columna **Churn Status** son missing values (NaN).

La compañía nos pide que proporcionemos los IDs de los 100 clientes de la base de datos de explotación que con mayor probabilidad vayan a abandonar la compañía. Para ello proporcionaremos como entregable a la compañía un archivo csv con una sola columna, **Customer ID**, y 100 filas que contengan los IDs de los clientes seleccionados.

El fichero **ejemplo_fichero_predicciones.csv** contiene un ejemplo con el formato solicitado para este archivo.

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import random
from statistics import mean


In [2]:
datos_construccion = pd.read_csv("./data/fuga_clientes_empresa_telefonica_construccion.csv")
datos_explotacion  = pd.read_csv("./data/fuga_clientes_empresa_telefonica_explotacion.csv")

In [3]:
datos_construccion.columns

Index(['Customer ID', 'network_age', 'Customer tenure in month',
       'Total Spend in Months 1 and 2', 'Total SMS Spend', 'Total Data Spend',
       'Total Data Consumption', 'Total Unique Calls', 'Total Onnet spend',
       'Total Offnet spend', 'Total Call centre complaint calls',
       'Churn Status', 'Most Loved Competitor network in Month 1_0',
       'Most Loved Competitor network in Month 1_Mango',
       'Most Loved Competitor network in Month 1_PQza',
       'Most Loved Competitor network in Month 1_ToCall',
       'Most Loved Competitor network in Month 1_Uxaa',
       'Most Loved Competitor network in Month 1_Weematel',
       'Most Loved Competitor network in Month 1_Zintel',
       'Most Loved Competitor network in Month 2_Mango',
       'Most Loved Competitor network in Month 2_PQza',
       'Most Loved Competitor network in Month 2_ToCall',
       'Most Loved Competitor network in Month 2_Uxaa',
       'Most Loved Competitor network in Month 2_Weematel',
       'Most 

In [4]:
datos_explotacion.columns

Index(['Customer ID', 'network_age', 'Customer tenure in month',
       'Total Spend in Months 1 and 2', 'Total SMS Spend', 'Total Data Spend',
       'Total Data Consumption', 'Total Unique Calls', 'Total Onnet spend',
       'Total Offnet spend', 'Total Call centre complaint calls',
       'Churn Status', 'Most Loved Competitor network in Month 1_0',
       'Most Loved Competitor network in Month 1_Mango',
       'Most Loved Competitor network in Month 1_PQza',
       'Most Loved Competitor network in Month 1_ToCall',
       'Most Loved Competitor network in Month 1_Uxaa',
       'Most Loved Competitor network in Month 1_Weematel',
       'Most Loved Competitor network in Month 1_Zintel',
       'Most Loved Competitor network in Month 2_Mango',
       'Most Loved Competitor network in Month 2_PQza',
       'Most Loved Competitor network in Month 2_ToCall',
       'Most Loved Competitor network in Month 2_Uxaa',
       'Most Loved Competitor network in Month 2_Weematel',
       'Most 

In [5]:
# chequeo de que las columnas son exactamente las mismas en los dos ficheros:
sum(datos_construccion.columns != datos_explotacion.columns)

0

In [6]:
# la columna a predecir es "Churn Status"
# si es 1, el cliente se va de la compañía
# si es 0, el cliente se queda

# Esta columna se sabe en el dataset de construcción (ejemplos de clientes pasados):
datos_construccion["Churn Status"].values[:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1])

In [7]:
# Sin embargo no se sabe en el dataset de explotación (clientes actuales):
datos_explotacion["Churn Status"].values[:10]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

#### Posibles pasos a realizar:
1. Tratado de los datos construccion
2. Churn Status (valor a predecir)
3. Red neuronal, busqueda de las capas y neuronas optimas
4. entrenamiento de la red y testeo
5. prediccion de explotacion

### Seleccionando los datos
Para empezar descartaremos el "Customer ID" porque no tiene mucha relevancia a nivel estadístico y meterémos los datos en varibles separandolos.

In [47]:

atrs = list(datos_construccion.columns)

atrs.remove("Churn Status")

atrs.remove("Customer ID")
print(atrs)

X_c = datos_construccion[atrs].values
y_c = datos_construccion["Churn Status"].values

['network_age', 'Customer tenure in month', 'Total Spend in Months 1 and 2', 'Total SMS Spend', 'Total Data Spend', 'Total Data Consumption', 'Total Unique Calls', 'Total Onnet spend', 'Total Offnet spend', 'Total Call centre complaint calls', 'Most Loved Competitor network in Month 1_0', 'Most Loved Competitor network in Month 1_Mango', 'Most Loved Competitor network in Month 1_PQza', 'Most Loved Competitor network in Month 1_ToCall', 'Most Loved Competitor network in Month 1_Uxaa', 'Most Loved Competitor network in Month 1_Weematel', 'Most Loved Competitor network in Month 1_Zintel', 'Most Loved Competitor network in Month 2_Mango', 'Most Loved Competitor network in Month 2_PQza', 'Most Loved Competitor network in Month 2_ToCall', 'Most Loved Competitor network in Month 2_Uxaa', 'Most Loved Competitor network in Month 2_Weematel', 'Most Loved Competitor network in Month 2_Zintel']


Se deben normalizar los datos

In [28]:
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X_c, y_c, train_size=0.8, random_state=0)
scaler = MinMaxScaler((0,1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.feature_selection import SelectKBest
select = SelectKBest(k=23)

X_train_selected = select.fit_transform(X_train, y_train)
X_test_selected = select.transform(X_test)


Una vez se han separado estos valores toca elegir que estimador puede sernos más útil para este conjunto de datos.

In [10]:
def train_n_check(grid_param, estimator):

    k_tests = RepeatedStratifiedKFold(n_splits=10, n_repeats=5) # Repite tests
    grid_search = GridSearchCV(estimator(), grid_param, cv=k_tests, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    
    param_names =  list(grid_param.keys())
    print("Estimación de grid search:")
    for param_name in param_names: 
        print("- "+param_name +" = ", str(grid_search.best_params_[param_name]))
    return grid_search.best_params_
import random

def random_create_n_select_layers(min_neurons, min_layers, max_neurons, max_layers, num_combinations):
    '''
    Crea aleatoriamente un conjunto entre unas neuronas y capas dadas
        (se ha de proporcionar también el número de combinaciones totales).
    '''
    combinations = [[(j,)*i for j in range(min_neurons, max_neurons+1)] for i in range(min_layers, max_layers+1)]
    
    each_layer = int(num_combinations/(max_layers-min_layers+1))

    final =list()

    for i in range(max_layers):
            prov_list = combinations[i][: (max_neurons-min_neurons+1)]
            random.shuffle(prov_list)
            final+=(prov_list[:each_layer])
        
    return final
            

### Naïve Bayes

In [11]:
smooth_pow = 100
grid_param= {"var_smoothing": [i**(-i) for i in range(1,smooth_pow)]}
best_params = train_n_check(grid_param, GaussianNB)

clf = GaussianNB(var_smoothing= best_params['var_smoothing']) 
scores = cross_val_score(clf, X_train_selected, y_train, cv=10) # 5-fold cross-validation


print('Precisión en cada una de las particiones: ', scores)
print('Estimación de la precisión por validación cruzada: {:.2f} +/- {:.2f}'.format(scores.mean(), scores.std()))

Estimación de grid search:
- var_smoothing =  2.143347050754458e-05
Precisión en cada una de las particiones:  [0.59550562 0.6741573  0.73033708 0.6741573  0.66292135 0.68539326
 0.74157303 0.75280899 0.68181818 0.64772727]
Estimación de la precisión por validación cruzada: 0.68 +/- 0.04


### Arbol de Decisión

In [14]:
max_max_depth = 100
grid_param= {"max_depth": [i for i in range(1,max_max_depth)]}
best_params = train_n_check(grid_param, DecisionTreeClassifier)

clf = DecisionTreeClassifier(max_depth= best_params['max_depth'])
scores = cross_val_score(clf, X_train_selected, y_train, cv=10) # 5-fold cross-validation

print('Precisión en cada una de las particiones: ', scores)
print('Estimación de la precisión por validación cruzada: {:.2f} +/- {:.2f}'.format(scores.mean(), scores.std()))

Estimación de grid search:
- max_depth =  2
Precisión en cada una de las particiones:  [0.78651685 0.79775281 0.7752809  0.75280899 0.74157303 0.7752809
 0.78651685 0.7752809  0.79545455 0.73863636]
Estimación de la precisión por validación cruzada: 0.77 +/- 0.02


### Knn Neighbors

In [11]:
max_knn = 100
grid_param= {"n_neighbors": [i for i in range(1,max_knn)]}
best_params = train_n_check(grid_param, KNeighborsClassifier)

clf = KNeighborsClassifier(n_neighbors=best_params['n_neighbors']) 
scores = cross_val_score(clf, X_train_selected, y_train, cv=10) # 5-fold cross-validation

print('Precisión en cada una de las particiones: ', scores)
print('Estimación de la precisión por validación cruzada: {:.2f} +/- {:.2f}'.format(scores.mean(), scores.std()))

Estimación de grid search:
- n_neighbors =  16
Precisión en cada una de las particiones:  [0.65168539 0.66292135 0.68539326 0.71910112 0.64044944 0.66292135
 0.74157303 0.76404494 0.65909091 0.57954545]
Estimación de la precisión por validación cruzada: 0.68 +/- 0.05


### Regresión logística

In [74]:
tol_pow = 10
tol_list= [i**(-i) for i in range(1,tol_pow)]

grid_param= {"tol":tol_list, "solver":["lbfgs"], "max_iter": [1000], "penalty":["l2"]}
best_params = train_n_check(grid_param, LogisticRegression)

clf = LogisticRegression(penalty=best_params['penalty'],
                         tol=best_params['tol'],
                         max_iter=best_params['max_iter'],
                        solver=best_params["solver"]) 
scores = cross_val_score(clf, X_train_selected, y_train, cv=10) # 5-fold cross-validation

print('Precisión en cada una de las particiones: ', scores)
print('Estimación de la precisión por validación cruzada: {:.2f} +/- {:.2f}'.format(scores.mean(), scores.std()))

Estimación de grid search:
- tol =  0.037037037037037035
- solver =  lbfgs
- max_iter =  1000
- penalty =  l2
Precisión en cada una de las particiones:  [0.64044944 0.6741573  0.73033708 0.66292135 0.66292135 0.66292135
 0.71910112 0.7752809  0.67045455 0.625     ]
Estimación de la precisión por validación cruzada: 0.68 +/- 0.04


### Red Neuronal

In [13]:

max_neurons = 5

hidden_layer_sizes = [(10*i,) for i in range(1, max_neurons+1)]

grid_param= {"max_iter": [9000], "hidden_layer_sizes":hidden_layer_sizes,
             "alpha":[0.0]}

best_params = train_n_check(grid_param, MLPClassifier)
optimal_neurons = best_params["hidden_layer_sizes"]

scores = cross_val_score(clf, X_train_selected, y_train, cv=10) # 5-fold cross-validation

print('Precisión en cada una de las particiones: ', scores)
print('Estimación de la precisión por validación cruzada: {:.2f} +/- {:.2f}'.format(scores.mean(), scores.std()))

Estimación de grid search:
- max_iter =  9000
- hidden_layer_sizes =  (40,)
- alpha =  0.0
Precisión en cada una de las particiones:  [0.64044944 0.6741573  0.73033708 0.6741573  0.66292135 0.6741573
 0.71910112 0.7752809  0.67045455 0.625     ]
Estimación de la precisión por validación cruzada: 0.68 +/- 0.04


In [16]:

min_neurons = optimal_neurons[0] 
max_neurons = optimal_neurons[0] + 2
min_layers = 1
max_layers = 5

hidden_layer_sizes = random_create_n_select_layers(min_neurons,min_layers,max_neurons,max_layers, 10)

grid_param= {"max_iter": [9000], "hidden_layer_sizes":hidden_layer_sizes,
             "alpha":[0.0]}

best_params = train_n_check(grid_param, MLPClassifier)

clf = MLPClassifier(hidden_layer_sizes=best_params['hidden_layer_sizes'],
                    max_iter=best_params['max_iter'],
                    alpha=best_params['alpha']) 

scores = cross_val_score(clf, X_train_selected, y_train, cv=5) # 5-fold cross-validation

print('Precisión en cada una de las particiones: ', scores)
print('Estimación de la precisión por validación cruzada: {:.2f} +/- {:.2f}'.format(scores.mean(), scores.std()))


Estimación de grid search:
- max_iter =  9000
- hidden_layer_sizes =  (40,)
- alpha =  0.0
Precisión en cada una de las particiones:  [0.71910112 0.70786517 0.74157303 0.75706215 0.70056497]
Estimación de la precisión por validación cruzada: 0.73 +/- 0.02


Tras estas comprobaciones parece que se puede llegar a obtener mejor predictibilidad con los decision trees, aun así se va a realizar tanto con redes neuronales como con decision trees la siguiente prueba.


In [36]:
scores_nn = list()
scores_dt = list()

for i in range(40):
    clf = MLPClassifier(hidden_layer_sizes=(40,), max_iter=3000,alpha=0.0)
    clf.fit(X_train_selected, y_train)
    y_pred = clf.predict(X_test_selected)
    scores_nn.append(accuracy_score(y_test, y_pred))
    
    clf = DecisionTreeClassifier(max_depth=2)
    clf.fit(X_train_selected, y_train)
    y_pred = clf.predict(X_test_selected)
    scores_dt.append(accuracy_score(y_test, y_pred))

print("Redes neuronales:", mean(scores_nn))
print("Arboles decision:", mean(scores_dt))

Redes neuronales: 0.6734234234234234
Arboles decision: 0.7252252252252253


Como se puede ver los árboles de decisión predicen mejor este conjubto de datos, seguramente por que no hay datos suficientes como para que la red neuronal de valores lo suficientemente precisos.

In [82]:
import csv

X_ex = datos_explotacion[atrs].values
ids = datos_explotacion['Customer ID'].values

clf = DecisionTreeClassifier(max_depth=2)
clf.fit(X_train_selected, y_train)
y_ex = clf.predict(X_ex)

predicted = list()
for i in range (100):
    predicted.append(ids[i])


print(ids) 

# a csv
df = pd.DataFrame()
df['Customer_ID'] = ids
df.to_csv("file_predictions.csv", index=False)      


['ADF1330' 'ADF1331' 'ADF1345' 'ADF1349' 'ADF1363' 'ADF1372' 'ADF1403'
 'ADF1404' 'ADF1410' 'ADF1425' 'ADF1429' 'ADF1431' 'ADF1433' 'ADF1438'
 'ADF1439' 'ADF1445' 'ADF1450' 'ADF1452' 'ADF1453' 'ADF1456' 'ADF1462'
 'ADF1465' 'ADF1477' 'ADF1481' 'ADF1489' 'ADF1502' 'ADF1503' 'ADF1521'
 'ADF1554' 'ADF1559' 'ADF1560' 'ADF1561' 'ADF1562' 'ADF1563' 'ADF1564'
 'ADF1567' 'ADF1582' 'ADF1590' 'ADF1596' 'ADF1597' 'ADF1598' 'ADF1601'
 'ADF1602' 'ADF1603' 'ADF1604' 'ADF1606' 'ADF1607' 'ADF1614' 'ADF1615'
 'ADF1616' 'ADF1617' 'ADF1620' 'ADF1623' 'ADF1624' 'ADF1625' 'ADF1632'
 'ADF1634' 'ADF1635' 'ADF1638' 'ADF1639' 'ADF1644' 'ADF1650' 'ADF1656'
 'ADF1657' 'ADF1663' 'ADF1668' 'ADF1670' 'ADF1677' 'ADF1678' 'ADF1685'
 'ADF1695' 'ADF1698' 'ADF1707' 'ADF1708' 'ADF1719' 'ADF1721' 'ADF1725'
 'ADF1726' 'ADF1728' 'ADF1729' 'ADF1734' 'ADF1736' 'ADF1754' 'ADF1759'
 'ADF1760' 'ADF1762' 'ADF1763' 'ADF1767' 'ADF1772' 'ADF1773' 'ADF1774'
 'ADF1803' 'ADF1806' 'ADF1811' 'ADF1815' 'ADF1817' 'ADF1821' 'ADF1841'
 'ADF1